# RSA Cryptography Lab
<p>
    This notebook will pertain research and information regarding RSA and EAS cryptography in order to set up a basic lab environment in which a hybrid system cipher is formed. From this point I will be able to utilize the lab environment by trying different variations of Shor's Algorithm in qiskit in order to crack the cipher and prove RSA is an unsafe protocol in the modern day for those who want protection from serious cryptanalyst's
</p>

### Importing necessary modules

In [9]:
from Cryptodome.Cipher import AES, PKCS1_OAEP
from Cryptodome.PublicKey import RSA
from Cryptodome.Random import get_random_bytes
from io import BytesIO
import base64
import zlib

### Creating a function to generate RSA keys

In [13]:
def generate():
    new_key = RSA.generate(2048)
    private_key = new_key.exportKey()
    public_key = new_key.publickey().exportKey()

    with open("key.pri", "wb") as f:
        f.write(private_key)

    with open("key.pub", "wb") as f:
        f.write(public_key)

### Creating a function to grab either public or private key

In [14]:
def get_rsa_cipher(keytype):
    with open(f"key.{keytype}") as f:
        key = f.read()
    rsakey = RSA.importKey(key)
    return (PKCS1_OAEP.new(rsakey), rsakey.size_in_bytes())

### Creating an encryption function
This function encrypts data using RSA and EAS to form a hybrid encryption system.

In [15]:
def encrypt(plaintext):
    compressed_text = zlib.compress(plaintext)

    session_key = get_random_bytes(16)
    cipher_aes = AES.new(session_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(compressed_text)

    cipher_rsa, _ = get_rsa_cipher("pub")
    encrypted_session_key = cipher_rsa.encrypt(session_key)

    msg_payload = encrypted_session_key + cipher_aes.nonce + tag + ciphertext
    encrypted = base64.encodebytes(msg_payload)
    return encrypted

### Creating a decryption function
This function will encrypt the data out of this hybrid cipher. I can use this to check if my own decipherment of the keys using Shor's Algorithm worked correctly.

In [16]:
def decrypt(encrypted):
    encrypted_bytes = BytesIO(base64.decodebytes(encrypted))
    cipher_rsa, keysize_in_bytes = get_rsa_cipher("pri")

    encrypted_session_key = encrypted_bytes.read(keysize_in_bytes)
    nonce = encrypted_bytes.read(16)
    tag = encrypted_bytes.read(16)
    ciphertext = encrypted_bytes.read()

    session_key = cipher_rsa.decrypt(encrypted_session_key)
    cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
    decrypted = cipher_aes.decrypt_and_verify(ciphertext, tag)

    plaintext = zlib.decompress(decrypted)
    return plaintext

### Generate keys for RSA
This segment acts like main would and generates keys for RSA and EAS

In [17]:
generate()

### Encrypt actual plaintext
This contains the value for the plaintext string and prints back the encrypted text

In [18]:
plaintext = b"Two can keep a secret."
print(encrypt(plaintext))

b'EE27kLf3V1+RiNAzuLu3ho7MCyxTKjo/NBTFr4/ZsQB+7fmZKCEcyhmHHSML69MEBmVPTjzawma4\njX6QjGDRBEcxSpSxBaLnip+0+DFAkRU2SVzWkL1LgjlzEVm6/+1xODDaW9fRRWJFJekTpxH6Zp+K\n6qxyh4Y7gTz7RX439x+kedNFecVe0Ykeq2hTSSdgZPNjwN148pBiHc1QbvMPWEvN7FUu8AgXSjd0\nV3oNpW17HW9xLYCjUfjLp58QMWh9T6BN6nsGv8HP4NebdnGtLEpeBjHzcFymZtqiT4aWigwach+b\nkqvXfy/VfoDHekzadW2LMkBCkFsylBCXXpMe6msrokCwBx/HXtasNbwEvdx2FjZLcc/WijexteM5\nAfs9o3FwiSgKX72h+WbSu0TaoRZcwnCFPwmmPc53NlGJ\n'


### Show decrypted text

In [19]:
print(decrypt(encrypt(plaintext)))

b'Two can keep a secret.'


## Going forward with this environment...
I think the best way to go forth from here woul be to test different Qiskit implementations of Shor's Algorithm to try and decrypt the RSA key for this lab environment. We can verify by attempting to decrypt using the key generated from our Shor implementation then compare that to the value we know the string to be which is printed above.